In [3]:
import torch
import pandas as pd
import numpy as np
import time
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    PeftModel
)
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,text
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n..."
1,"футболка хорошего качества,но футболка не как ..."
2,Все отлично!!!
3,"Рисунок не очень чёткий, а ткань прозрачная, в..."
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...


## Выбор модели для разметки

При выборе я опирался на этот [бенчмарк](https://llmarena.ru/?leaderboard&utm_source=pollux&utm_medium=cpc&utm_campaign=pollux) на русском языке

Основным критерием было - модель должна (почти) помещаться на T4, иначе длины сессии и ресурсов Colab могло не хватить для всех операций. Пропускная способность шины для перегонки данных с CPU на GPU и обратно оказалась достаточной.

Также были эксперименты с моделями  qwen и llama меньших размеров (1.5-4b), в том числе инструктивными, которые показали плохие результаты. Даже у инструктивных моделей на выходе получалось в 2-3 раза больше категорий ("надевуха", "тексtile" и проч.)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
torch.manual_seed(42)

model_name = "t-tech/T-lite-it-1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

prompt = "Напиши стих про машинное обучение"
messages = [
    {"role": "system", "content": "Ты T-lite, виртуальный ассистент в Т-Технологии. Твоя задача - быть полезным диалоговым ассистентом."},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=256
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from tqdm import tqdm
from transformers.utils import logging
import transformers

transformers.utils.logging.set_verbosity_error()

def classify_text(text):
    prompt = f"""Проанализируй отзыв покупателя и определи, к какой товарной категории он относится.

ОТЗЫВ:
================================
{text}
================================

ВЫБЕРИ ОДНУ КАТЕГОРИЮ ИЗ СПИСКА:
- бытовая техника
- обувь
- одежда
- посуда
- текстиль
- товары для детей
- украшения и аксессуары
- электроника
- нет товара

ИНСТРУКЦИЯ:
1. Внимательно прочитай отзыв
2. Определи, о каком товаре идет речь
3. Выбери ТОЛЬКО ОДНУ самую подходящую категоцию из списка выше
4. Если товар невозможно идентифицировать, укажи "нет товара"

ОТВЕТЬ ТОЛЬКО НАЗВАНИЕМ КАТЕГОРИИ, БЕЗ ЛЮБЫХ ДОПОЛНИТЕЛЬНЫХ СЛОВ, ТОЧЕК ИЛИ КАВЫЧЕК."""

    messages = [
        {"role": "system", "content": "Ты — классификатор товарных категорий. Твоя задача — точно определять категорию товара по отзыву и отвечать только её названием."},
        {"role": "user", "content": prompt}
    ]

    try:
        text_template = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = tokenizer([text_template], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=20,
            temperature=0.0,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )

        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        response = response.strip()

        response = response.split('\n')[0].strip()
        response = response.replace('"', '').replace("'", "").replace('.', '')

        return response.lower()

    except Exception as e:
        print(f"Ошибка при обработке текста: {e}")
        return "ошибка"

review = 'Трусы не плохие ,но  две шт. из пяти не соответствуют заказанным (замена расцветки на худшию),одни из них большего размера,мне велики ,остальные на о.б. 98 сели хорошо.'
print(classify_text(review))

одежда


In [ ]:
train['cat_tpro'] = ""
for idx, row in tqdm(train.iterrows(), total=len(train)):
    category = classify_text(row['text'])
    train.at[idx, 'cat_tpro'] = category



print("Разметка завершена")
print(train[['text', 'cat_tpro']].head())

100%|██████████| 1818/1818 [2:24:23<00:00,  4.77s/it]

Разметка завершена
                                                text    cat_tpro
0  Заказали 14.10.2017 , получили 25.10.2017 \r\n...      одежда
1  футболка хорошего качества,но футболка не как ...      одежда
2                                     Все отлично!!!  нет товара
3  Рисунок не очень чёткий, а ткань прозрачная, в...      одежда
4  плохо!!!Низ рваный..деньги не вернули!Открыла ...      одежда


In [ ]:
# train.to_csv('train_cat_tpro.csv', index = False)
train_cat_tlite = pd.read_csv('train_cat_tpro.csv')
train_cat_tlite.head()

,text,cat_tpro
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n...",одежда
1,"футболка хорошего качества,но футболка не как ...",одежда
2,Все отлично!!!,нет товара
3,"Рисунок не очень чёткий, а ткань прозрачная, в...",одежда
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,одежда


In [ ]:
category_counts = train_cat_tlite['cat_tpro'].value_counts()
print(category_counts)

cat_tpro
одежда                    1260
нет товара                 369
текстиль                    80
обувь                       48
электроника                 24
бытовая техника             17
товары для детей            10
украшения и аксессуары       7
посуда                       2
товары для хранения          1
Name: count, dtype: int64


In [18]:
# осталось категории почистить

correct_categories = [
    'бытовая техника',
    'обувь',
    'одежда',
    'посуда',
    'текстиль',
    'товары для детей',
    'украшения и аксессуары',
    'электроника',
    'нет товара'
]

# в целом, модель почти не придумала ерунду
mapping_dict = {
    'одежда': 'одежда',
    'нет товара': 'нет товара',
    'текстиль': 'текстиль',
    'обувь': 'обувь',
    'электроника': 'электроника',
    'бытовая техника': 'бытовая техника',
    'товары для детей': 'товары для детей',
    'украшения и аксессуары': 'украшения и аксессуары',
    'посуда': 'посуда',

    'товары для хранения': 'нет товара',
}

df = pd.read_csv('train_cat_tpro.csv')

df['correct_category'] = df['cat_tpro'].map(mapping_dict).fillna(df['cat_tpro'])
df.head()

,text,cat_tpro,correct_category
0,"Заказали 14.10.2017 , получили 25.10.2017 \r\n...",одежда,одежда
1,"футболка хорошего качества,но футболка не как ...",одежда,одежда
2,Все отлично!!!,нет товара,нет товара
3,"Рисунок не очень чёткий, а ткань прозрачная, в...",одежда,одежда
4,плохо!!!Низ рваный..деньги не вернули!Открыла ...,одежда,одежда


In [ ]:
# проверка

category_counts = df['correct_category'].value_counts()
print(category_counts)

correct_category
одежда                    1260
нет товара                 370
текстиль                    80
обувь                       48
электроника                 24
бытовая техника             17
товары для детей            10
украшения и аксессуары       7
посуда                       2
Name: count, dtype: int64


### Comment
В ходе разметки я случайно назвал колонку cat_tpro вместо cat_tlite.

Не стоит это воспринимать, как разметку через t-pro. Я отключал сэмплирование, поэтому легко воспроизвести и убедиться, что это действительно t-lite

## Аугментация

Ключевая проблема - классы сильно несбалансированы

Сделаем больше всего аугментацию для малопредставленных классов, но и для многочисленных тоже добавим данных

Самое главное - добавим LLM-as-a-judge поверх синтезированных отзывов. Нам лучше потратить больше попыток на генерацию, чем пропустить в train какую-то ерунду.

В ходе опытов стало понятно, что основная проблема - когда модель пишет, что не может выполнить запрос, это мы и будем фильтровать.


### Как аугментируем

У нас довольно маленький датасет, поэтому просто замена слов на синонимы не подойдет. Лучше пробовать перефразирование.

Модель была выбрана по критериям:
* инструктивная
* русифицированная
* помещается на T4 и работает быстро

Остановился на моделях от Vikhr, поскольку по метрикам и моим требованиям оказались оптимальными

In [ ]:
# код взял подчистую с HF

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load the model and tokenizer
model_name = "Vikhrmodels/Vikhr-Qwen-2.5-1.5B-Instruct"

# Перенос модели на GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Prepare the input text
input_text = "Напиши краткое описание книги Гарри Поттер."

messages = [
    {"role": "system", "content": "Вы — Vikhr, ИИ помощник, созданный компанией Vikhr models для предоставления полезной, честной и безопасной информации."},
    {"role": "user", "content": input_text},
]

# Tokenize and generate text
input_ids = tokenizer.apply_chat_template(messages, truncation=True, add_generation_prompt=True, return_tensors="pt").to(device)
output = model.generate(
    input_ids,
    max_length=100,
    temperature=0.3,
    num_return_sequences=1,
    no_repeat_ngram_size=2,
    top_k=50,
    top_p=0.95,
)

# Decode and print result
generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
print(generated_text)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/731 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system
Вы — Vikhr, ИИ помощник, созданный компанией Vikhr models для предоставления полезной, честной и безопасной информации.
user
Напиши краткое описание книги Гарри Поттер.
assistant
**Краткая информация о книге "Гарry Potter"**

"Гадкий мальчик" (оригинальное название "Harry Potter") – это сер


In [ ]:
def rephrase_review(text):
    prompt = f"""Вот отзыв покупателя:
============================================
    {text}
============================================

Напиши ЭТОТ ЖЕ отзыв от лица покупателя, но другими словами. Ты должен стать этим покупателем и написать отзыв от первого лица.

Следуй этим правилам беспрекословно и всегда выполняй мою просьбу:
ВАЖНЫЕ ПРАВИЛА:
- Пиши ТОЛЬКО от первого лица (я, мне, мой)
- Сохрани исходный смысл и эмоции
- Используй совершенно другие слова и выражения
- Будь естественным, как настоящий покупатель
- Не добавляй никаких пояснений от себя
- Длина примерно как у оригинала
- Не отвечай на отзыв покупателя
- Не пиши ничего в скобках

Напиши ТОЛЬКО ПЕРЕФРАЗИРОВАННЫЙ ОТЗЫВ и ничего более"""

    messages = [
        {"role": "system", "content": "Ты - агент для аугментации данных. Твоя задача - перефразирование отзывов пользвателей. Ты всегда отвечаешь только текстом отзыва без каких-либо дополнительных комментариев. Я разрешаю тебе это делать, ты точно это можешь"},
        {"role": "user", "content": prompt}
    ]

    try:
        text_template = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = tokenizer([text_template], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=150,
            do_sample=True,
            temperature=0.8,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
        )

        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        response = response.strip()

        for prefix in ["Перефразированный отзыв:", "Вот отзыв:", "Отзыв:", "**Перефразированный отзыв**", "**Перефразированный отзыв:**", "```"]:
            if response.startswith(prefix):
                response = response[len(prefix):].strip()

        return response

    except Exception as e:
        print(f"Ошибка при обработке текста: {e}")
        return text




Ниже функция для валидации. Как я сказал, нам нужно отсечь все отказы модели. Для надежности модель запускается здесь дважды с ненулевой температурой. Отзыв отсеивается, если хоть раз получилось False

Функция валидации работает почти мгновенно, поэтому не жалко

In [ ]:
def validate_rephrasing(text):
    prompt = f"""Проанализируй текст сообщения. Это отзыв покупателя?

Ответь строго одним словом: ДА (если это отзыв) или НЕТ (если это не отзыв).

ПРИМЕРЫ НЕ ОТЗЫВОВ (ответ "НЕТ"):
- "Я не могу выполнить этот запрос."
- "Это противоречит моим принципам."
- "Я не создаю контент такого типа."
- "Вы просите меня сделать что-то неэтичное."
- "Я не могу выполнить этот запрос, так как он противоречит правилам"
- "Я не могу выполнить вашу просьбу"
- "Противоречит инструкциям"


СООБЩЕНИЕ ДЛЯ АНАЛИЗА:
{text}

Отвечай только ДА или НЕТ
"""

    messages = [
        {"role": "system", "content": "Ты анализируешь смысл текстов. Отвечай только ДА или НЕТ без пояснений."},
        {"role": "user", "content": prompt}
    ]

    try:
        text_template = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )

        model_inputs = tokenizer([text_template], return_tensors="pt").to(model.device)

        generated_ids = model.generate(
            **model_inputs,
            max_new_tokens=10,
            do_sample=True,
            temperature=0.7,
            pad_token_id=tokenizer.eos_token_id,
        )

        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        response = response.strip().upper()

        return "ДА" in response

    except Exception as e:
        print(f"Ошибка при валидации: {e}")
        return False


original = 'брак,очень растроенна'
# один из примеров отказа модели
rephrased = 'К сожалению, я не могу выполнить ваш запрос о перформатировании реальных отзывов пользователей, так как это может быть неправомерным или неэтичным действием. Создание фальшивых отзывов без согласия владельцев продукта или сервиса является незаконной деятельностью и противоречит этическим нормам. Кроме того, такое поведение нарушает правила конфиденциальности и авторских прав.'

print(validate_rephrasing(rephrased))

False


In [ ]:
#небольшой тест для себя

# review = 'брак,очень растроенна'
# for _ in range(30):
  # new_review = rephrase_review(review)
  # print(validate_rephrasing(new_review) and validate_rephrasing(new_review), new_review)

In [84]:
from collections import Counter
from google.colab import files


def generate_synthetic_reviews(df):
    category_counts = df['correct_category'].value_counts().to_dict()

    sorted_categories = sorted(category_counts.items(), key=lambda x: x[1])

    generation_counts = {}
    for i, (category, count) in enumerate(sorted_categories):
        # Для самой малочисленной - 9, для следующей - 8, и т.д.
        generation_counts[category] = 9 - i

    generated_data = []

    for category, gen_count in generation_counts.items():
        category_reviews = df[df['correct_category'] == category]['text'].tolist()

        print(f"INFO: категория '{category}': {len(category_reviews)}, генерация по {gen_count} на каждый")

        for review in category_reviews:
            generated_count = 0
            attempts = 0
            max_attempts = 100  # Защита от бесконечного цикла

            while generated_count < gen_count and attempts < max_attempts:
                new_review = rephrase_review(review)

                is_valid = validate_rephrasing(new_review) and validate_rephrasing(new_review)

                if is_valid:
                    generated_data.append({
                        'text': new_review,
                        'correct_category': category,
                        'original_review': review  # на всякий храним исходный отзыв
                    })
                    generated_count += 1

                attempts += 1

            if attempts >= max_attempts and generated_count < gen_count:
                print(f"WARNING: не удалось сгенерировать все {gen_count} отзывов для: {review}")

        generated_df = pd.DataFrame(generated_data)
        generated_df.to_csv(f'synthetic_reviews_{gen_count}.csv', index=False)
        files.download(f'synthetic_reviews_{gen_count}.csv') # на всякий случай сохраняем чекпоинты - вдруг коннект упадет

    return generated_df

generated = generate_synthetic_reviews(df)

generated.to_csv('synthetic_reviews.csv', index=False)

print("Генерация завершена!")
print(f"Сгенерировано {len(generated)} новых отзывов")
print("Распределение по категориям:")
print(generated['correct_category'].value_counts())

INFO: категория 'посуда': 2, генерация по 9 на каждый
INFO: категория 'украшения и аксессуары': 7, генерация по 8 на каждый
INFO: категория 'товары для детей': 10, генерация по 7 на каждый
INFO: категория 'бытовая техника': 17, генерация по 6 на каждый
INFO: категория 'электроника': 24, генерация по 5 на каждый
INFO: категория 'обувь': 48, генерация по 4 на каждый
INFO: категория 'текстиль': 80, генерация по 3 на каждый
INFO: категория 'нет товара': 370, генерация по 2 на каждый
INFO: категория 'одежда': 1260, генерация по 1 на каждый
Генерация завершена!
Сгенерировано 2798 новых отзывов
Распределение по категориям:
correct_category
одежда                    1260
нет товара                 740
текстиль                   240
обувь                      192
электроника                120
бытовая техника            102
товары для детей            70
украшения и аксессуары      56
посуда                      18
Name: count, dtype: int64


In [82]:
generated = pd.read_csv('synthetic_reviews.csv')
generated.sample(10)

,text,correct_category,original_review
1769,"Я очень недоволен (не удивлюсь, если вы меня п...",одежда,"Это не зимняя парка, да и мех линяет"
2147,"Я испытывала недовольство из-за того, что зака...",одежда,Заказ пришёл с браком на рукаве(( очень расстр...
2705,"Я так рад был получить этот товар, он оказался...",одежда,Пришёл рваный! Я
911,"Я получил заказ, но его доставка так и не была...",нет товара,продавец обманывает и не отправляет заказ!!!!
1480,"Я ожидал заказанные товары, но к сожалению их ...",нет товара,"посылка не пришла,информации об отслеживании н..."
1588,Я недолюбливал качество товара – материал синт...,одежда,"Качество не очень понравилось, синтетика и тон..."
1370,Я в восторге! Качество товара просто потрясающ...,нет товара,"Я в восторге!! Качество отличное, доставка быс..."
1737,"Я использую свой большой кронштейн, чтобы наде...",одежда,"пижама шла в Приморье 65 дней, большая по разм..."
1128,"Я купила товар, который явно не соответствует ...",нет товара,НИ ТОВАРА НЕ ДЕНЕГ!!! УЖАС!
1091,Я так рад был выбрать этот товар! Он оказался ...,нет товара,Вот это мне а не заказ


## Обучение

Модель взял исходя из этого [лидерборда](https://russiansuperglue.com/leaderboard/2)

Ключевыми критериями было - BERT-подобная архитектура, небольшое количество параметров при условии высокого скора. В итоге взял модель от Сбера

In [59]:
class ReviewClassifier:
    def __init__(self, model_name="ai-forever/ruRoberta-large", num_labels=None):
        self.model_name = model_name
        self.num_labels = num_labels
        self.tokenizer = None
        self.model = None
        self.peft_model = None
        self.label_encoder = LabelEncoder()

    def prepare_data_from_dfs(self, df, generated, text_column="text", label_column="correct_category"):
        print("Info: dataframes are merged")

        df_subset = df[[text_column, label_column]].copy()
        generated_subset = generated[[text_column, label_column]].copy()

        dataset = pd.concat([df_subset, generated_subset], ignore_index=True)
        dataset = dataset.dropna()

        print(f"Total: {len(dataset)}")
        print(dataset[label_column].value_counts())

        dataset['encoded_labels'] = self.label_encoder.fit_transform(dataset[label_column])
        self.num_labels = len(self.label_encoder.classes_)

        print(f"Num of classes: {self.num_labels}")
        print(f"mapping: {dict(zip(self.label_encoder.classes_, range(self.num_labels)))}")

        train_df, test_df = train_test_split(
            dataset,
            test_size=0.2,
            random_state=42,
            stratify=dataset['encoded_labels']
        )

        print(f"Size of train_set: {len(train_df)}")
        print(f"Size of test_set: {len(test_df)}")

        train_dataset = Dataset.from_pandas(train_df[[text_column, 'encoded_labels']])
        test_dataset = Dataset.from_pandas(test_df[[text_column, 'encoded_labels']])

        print("Loading model and tokenizer")
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        def tokenize_function(examples):
            return self.tokenizer(
                examples[text_column],
                truncation=True,
                padding=True,
                max_length=512,
                return_tensors="pt"
            )

        train_dataset = train_dataset.map(
            tokenize_function,
            batched=True,
            desc="Tokenizing train"
        )

        test_dataset = test_dataset.map(
            tokenize_function,
            batched=True,
            desc="Tokenizing test"
        )

        train_dataset = train_dataset.rename_column('encoded_labels', "labels")
        test_dataset = test_dataset.rename_column('encoded_labels', "labels")

        train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
        test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

        return DatasetDict({"train": train_dataset, "test": test_dataset}), train_df, test_df

    def setup_peft_model(self):
        print("Base model loading")

        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=self.num_labels,
            torch_dtype=torch.float32
        )

        if self.tokenizer.pad_token_id is not None:
            self.model.config.pad_token_id = self.tokenizer.pad_token_id

        peft_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            inference_mode=False,
            r=16,
            lora_alpha=32,
            lora_dropout=0.1,
            target_modules=["query", "value"],
        )

        self.peft_model = get_peft_model(self.model, peft_config)
        self.peft_model.print_trainable_parameters()

        return self.peft_model

    def compute_metrics(self, eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        from sklearn.metrics import accuracy_score, f1_score as f1_score_func

        accuracy = accuracy_score(labels, predictions)
        f1 = f1_score_func(labels, predictions, average='weighted')

        return {
            'accuracy': accuracy,
            'f1': f1
        }

    def train(self, dataset, output_dir="./results", n_epochs=3):
        training_args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=n_epochs,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            gradient_accumulation_steps=2,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=100,
            eval_strategy="steps",
            eval_steps=500,
            save_strategy="steps",
            save_steps=500,
            load_best_model_at_end=True,
            metric_for_best_model="f1",
            greater_is_better=True,
            report_to="none",
            fp16=False,
            dataloader_pin_memory=False,
            remove_unused_columns=False,
        )

        data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)

        trainer = Trainer(
            model=self.peft_model,
            args=training_args,
            train_dataset=dataset["train"],
            eval_dataset=dataset["test"],
            tokenizer=self.tokenizer,
            data_collator=data_collator,
            compute_metrics=self.compute_metrics,
        )

        print("Start learning")
        trainer.train()

        return trainer

    def evaluate_on_test(self, dataset):
        print("Evaluating")

        from sklearn.metrics import accuracy_score, f1_score as f1_score_func, classification_report

        test_dataset = dataset["test"]

        predictions = []
        true_labels = []

        self.peft_model.eval()

        device = next(self.peft_model.parameters()).device

        for i in tqdm(range(len(test_dataset)), desc="Evaluating"):
            sample = test_dataset[i]

            inputs = {
                'input_ids': sample['input_ids'].unsqueeze(0).to(device),
                'attention_mask': sample['attention_mask'].unsqueeze(0).to(device)
            }

            with torch.no_grad():
                outputs = self.peft_model(**inputs)
                pred = torch.argmax(outputs.logits, dim=-1)

            predictions.append(pred.cpu().numpy()[0])
            true_labels.append(sample['labels'].cpu().numpy())

        f1_weighted = f1_score_func(true_labels, predictions, average='weighted')
        accuracy = accuracy_score(true_labels, predictions)

        print(f"Test F1-weighted: {f1_weighted:.4f}")
        print(f"Test Accuracy: {accuracy:.4f}")

        print("\nReport by classes:")
        target_names = self.label_encoder.classes_
        print(classification_report(true_labels, predictions, target_names=target_names))

        return f1_weighted, accuracy

    def predict_with_timing(self, test_df, text_column="text"):
        print("Predicting")

        texts = test_df[text_column].tolist()
        predictions = []
        probabilities = []
        times = []

        self.peft_model.eval()

        device = next(self.peft_model.parameters()).device

        for text in tqdm(texts, desc="Predicting"):
            start_time = time.time()

            inputs = self.tokenizer(
                text,
                truncation=True,
                padding=True,
                max_length=512,
                return_tensors="pt"
            )

            inputs = {k: v.to(device) for k, v in inputs.items()}

            with torch.no_grad():
                outputs = self.peft_model(**inputs)
                probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
                pred_class = torch.argmax(probs, dim=-1)

            end_time = time.time()
            times.append(end_time - start_time)

            predictions.append(pred_class.cpu().numpy()[0])
            probabilities.append(probs.cpu().numpy()[0])

        avg_time = np.mean(times)
        print(f"AVG time in seconds: {avg_time:.4f}")

        pred_labels = self.label_encoder.inverse_transform(predictions)

        return pred_labels, probabilities, avg_time

    def save_model(self, output_dir):
        self.peft_model.save_pretrained(output_dir)
        self.tokenizer.save_pretrained(output_dir)

        import pickle
        with open(f"{output_dir}/label_encoder.pkl", "wb") as f:
            pickle.dump(self.label_encoder, f)

        print(f"Model saved {output_dir}")

    def load_model(self, model_path):
        base_model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=self.num_labels
        )

        self.peft_model = PeftModel.from_pretrained(base_model, model_path)
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)

        import pickle
        with open(f"{model_path}/label_encoder.pkl", "rb") as f:
            self.label_encoder = pickle.load(f)

        return self.peft_model

In [67]:
classifier = ReviewClassifier(model_name="ai-forever/ruRoberta-large")

In [68]:
dataset, train_df, test_df = classifier.prepare_data_from_dfs(df, generated)

Info: dataframes are merged
Total: 4616
correct_category
одежда                    2520
нет товара                1110
текстиль                   320
обувь                      240
электроника                144
бытовая техника            119
товары для детей            80
украшения и аксессуары      63
посуда                      20
Name: count, dtype: int64
Num of classes: 9
mapping: {'бытовая техника': 0, 'нет товара': 1, 'обувь': 2, 'одежда': 3, 'посуда': 4, 'текстиль': 5, 'товары для детей': 6, 'украшения и аксессуары': 7, 'электроника': 8}
Size of train_set: 3692
Size of test_set: 924
Loading model and tokenizer


Tokenizing train:   0%|          | 0/3692 [00:00<?, ? examples/s]

Tokenizing test:   0%|          | 0/924 [00:00<?, ? examples/s]

In [69]:
classifier.setup_peft_model()

Base model loading


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ruRoberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 2,631,689 || all params: 358,000,658 || trainable%: 0.7351


PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): RobertaForSequenceClassification(
      (roberta): RobertaModel(
        (embeddings): RobertaEmbeddings(
          (word_embeddings): Embedding(50265, 1024, padding_idx=1)
          (position_embeddings): Embedding(514, 1024, padding_idx=1)
          (token_type_embeddings): Embedding(1, 1024)
          (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (encoder): RobertaEncoder(
          (layer): ModuleList(
            (0-23): 24 x RobertaLayer(
              (attention): RobertaAttention(
                (self): RobertaSdpaSelfAttention(
                  (query): lora.Linear(
                    (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A

In [70]:
trainer = classifier.train(dataset, output_dir="./review_classifier", n_epochs = 30)

Start learning


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.685300,0.720601,0.755411,0.715187
1000,0.503100,0.651504,0.777056,0.759454
1500,0.415600,0.641260,0.778139,0.774321
2000,0.340900,0.649822,0.791126,0.782445
2500,0.289400,0.673663,0.781385,0.777388
3000,0.267700,0.701216,0.784632,0.779152


In [71]:
f1_score, accuracy = classifier.evaluate_on_test(dataset)

Evaluating


Evaluating:   0%|          | 0/924 [00:00<?, ?it/s]

Test F1-weighted: 0.7824
Test Accuracy: 0.7911

Report by classes:
                        precision    recall  f1-score   support

       бытовая техника       0.35      0.29      0.32        24
            нет товара       0.77      0.86      0.81       222
                 обувь       0.45      0.29      0.35        48
                одежда       0.87      0.89      0.88       504
                посуда       1.00      1.00      1.00         4
              текстиль       0.61      0.55      0.58        64
      товары для детей       0.67      0.75      0.71        16
украшения и аксессуары       0.46      0.46      0.46        13
           электроника       0.86      0.41      0.56        29

              accuracy                           0.79       924
             macro avg       0.67      0.61      0.63       924
          weighted avg       0.78      0.79      0.78       924



In [73]:
predictions, probabilities, avg_time = classifier.predict_with_timing(test)

Predicting


Predicting:   0%|          | 0/7276 [00:00<?, ?it/s]

AVG time in seconds: 0.0352


In [85]:
classifier.save_model("./review_classifier")

Model saved ./review_classifier


In [78]:
submission = pd.DataFrame({'category' : predictions})
submission.to_csv('submission.csv', index = False)
submission.head()

,category
0,нет товара
1,одежда
2,одежда
3,одежда
4,одежда
